In [1]:
#### Basic Setting

import  load_dotenv
load_dotenv.load_dotenv("../../All_LLM_tutorial/.env")

True

In [2]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow

In [3]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

In [4]:
llm = OpenAI(model="gpt-4o-mini")

In [5]:
workflow = FunctionAgent(
    tools=[multiply, add],
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations using tools.",
)

In [6]:
response = await workflow.run(user_msg="What is 20+(2*4)?")
print(response)

The result of \( 20 + (2 \times 4) \) is 28.


# Tools

In [7]:
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

In [8]:
finance_tools = YahooFinanceToolSpec().to_tool_list()

In [9]:
finance_tools.extend([multiply, add])

In [10]:
workflow = FunctionAgent(
    name="Agent",
    description="Useful for performing financial operations.",
    llm=OpenAI(model="gpt-4o-mini"),
    tools=finance_tools,
    system_prompt="You are a helpful assistant.",
)


response = await workflow.run(
    user_msg="What's the current stock price of NVIDIA?"
)
print(response)

The current stock price of NVIDIA (NVDA) is $162.88.


# State

In [11]:
from llama_index.core.workflow import Context

In [20]:
async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.store.get("state")
    state["name"] = name
    await ctx.store.set("state", state)
    return f"Name set to {name}"

In [21]:
ctx_dict["state"]["state_data"]

{'_data': {'memory': '{"__is_component": true, "value": {"chat_store": {"store": {"chat_history": [{"role": "user", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Hi, my name is Laurie!"}]}, {"role": "assistant", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Hello Laurie! How can I assist you today?"}]}, {"role": "user", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "What\'s my name?"}]}, {"role": "assistant", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Your name is Laurie! How can I help you today, Laurie?"}]}]}, "class_name": "SimpleChatStore"}, "chat_store_key": "chat_history", "token_limit": 96000, "class_name": "ChatMemoryBuffer"}, "qualified_name": "llama_index.core.memory.chat_memory_buffer.ChatMemoryBuffer"}',
  'state': '{}',
  'formatted_input_with_state': 'false',
  'user_msg_str': '"What\'s my name?"',
  'scratchpad': '[]',
  'current_tool_calls': '[]'}}

In [22]:
ctx.store.get("state")

<coroutine object InMemoryStateStore.get at 0x7efc2dc84eb0>

In [23]:
workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

In [24]:
ctx = Context(workflow)

# check if it knows a name before setting it
response = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(str(response))

Your name has been set to "unset."


## How to check State!!!

In [25]:
ctx_dict = ctx.to_dict(serializer=JsonSerializer())
ctx_dict["state"]["state_data"]

{'_data': {'memory': '{"__is_component": true, "value": {"chat_store": {"store": {"chat_history": [{"role": "user", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Current state:\\n{\'name\': \'unset\'}\\n\\nCurrent message:\\nWhat\'s my name?\\n"}]}, {"role": "assistant", "additional_kwargs": {"tool_calls": [{"index": 0, "id": "call_fb8Okgq9sqpzH7Jm6dA1amwK", "function": {"arguments": "{\\"name\\":\\"unset\\"}", "name": "set_name"}, "type": "function"}]}, "blocks": [{"block_type": "text", "text": ""}]}, {"role": "tool", "additional_kwargs": {"tool_call_id": "call_fb8Okgq9sqpzH7Jm6dA1amwK"}, "blocks": [{"block_type": "text", "text": "Name set to unset"}]}, {"role": "assistant", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Your name has been set to \\"unset.\\""}]}]}, "class_name": "SimpleChatStore"}, "chat_store_key": "chat_history", "token_limit": 96000, "class_name": "ChatMemoryBuffer"}, "qualified_name": "llama_index.core.memory.chat_memory

In [27]:
response2 = await workflow.run(user_msg="My name is Laurie", ctx=ctx)
print(str(response2))

Your name has been updated to "Laurie."


In [28]:
ctx_dict = ctx.to_dict(serializer=JsonSerializer())
ctx_dict["state"]["state_data"]

{'_data': {'memory': '{"__is_component": true, "value": {"chat_store": {"store": {"chat_history": [{"role": "user", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Current state:\\n{\'name\': \'unset\'}\\n\\nCurrent message:\\nWhat\'s my name?\\n"}]}, {"role": "assistant", "additional_kwargs": {"tool_calls": [{"index": 0, "id": "call_fb8Okgq9sqpzH7Jm6dA1amwK", "function": {"arguments": "{\\"name\\":\\"unset\\"}", "name": "set_name"}, "type": "function"}]}, "blocks": [{"block_type": "text", "text": ""}]}, {"role": "tool", "additional_kwargs": {"tool_call_id": "call_fb8Okgq9sqpzH7Jm6dA1amwK"}, "blocks": [{"block_type": "text", "text": "Name set to unset"}]}, {"role": "assistant", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Your name has been set to \\"unset.\\""}]}, {"role": "user", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Current state:\\n{\'name\': \'unset\'}\\n\\nCurrent message:\\nMy name is Laurie\\n"}]}, {"role"

In [29]:
state = await ctx.store.get("state")
print("Name as stored in state: ", state["name"])

Name as stored in state:  Laurie


# Streaming?

In [30]:
from llama_index.tools.tavily_research import TavilyToolSpec
import os

In [31]:
tavily_tool = TavilyToolSpec(api_key=os.getenv("TAVILY_API_KEY"))

In [33]:
workflow = FunctionAgent(
    tools=tavily_tool.to_tool_list(),
    llm=llm,
    system_prompt="You're a helpful assistant that can search the web for information.",
)

In [34]:
from llama_index.core.agent.workflow import AgentStream

In [35]:
handler = workflow.run(user_msg="What's the weather like in San Francisco?")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

The current weather in San Francisco is generally mild. Here are some details:

- **Temperature**: The high is around 20°C (68°F) and the low is about 14°C (57°F).
- **Conditions**: The weather is partly cloudy with some sunny intervals.
- **Winds**: Winds are coming from the west at 10 to 20 mph.

For more detailed forecasts, you can check out [The Weather Channel](https://weather.com/weather/today/l/USCA0987:1:US) or [Weather25](https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=October).

# HITL

In [36]:
from llama_index.core.workflow import (
    InputRequiredEvent,
    HumanResponseEvent,
)

In [37]:
from llama_index.core.workflow import Context


async def dangerous_task(ctx: Context) -> str:
    """A dangerous task that requires human confirmation."""

    # emit a waiter event (InputRequiredEvent here)
    # and wait until we see a HumanResponseEvent
    question = "Are you sure you want to proceed? "
    response = await ctx.wait_for_event(
        HumanResponseEvent,
        waiter_id=question,
        waiter_event=InputRequiredEvent(
            prefix=question,
            user_name="Laurie",
        ),
        requirements={"user_name": "Laurie"},
    )

    # act on the input from the event
    if response.response.strip().lower() == "yes":
        return "Dangerous task completed successfully."
    else:
        return "Dangerous task aborted."

In [38]:
workflow = FunctionAgent(
    tools=[dangerous_task],
    llm=llm,
    system_prompt="You are a helpful assistant that can perform dangerous tasks.",
)

In [ ]:
handler = workflow.run(user_msg="I want to proceed with the dangerous task.")

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        # capture keyboard input
        response = input(event.prefix)
        # send our response back
        handler.ctx.send_event(
            HumanResponseEvent(
                response=response,
                user_name=event.user_name,
            )
        )

response = await handler
print(str(response))

Laurie
The dangerous task has been aborted. If you would like to proceed with it, please confirm your decision.


# Multi Agents

In [52]:
###Make tool for acting below tab
# Multi-Agent Tools
import json
from typing import Dict, Any

def search_web(query: str, ctx: Context) -> str:
    """웹에서 주어진 쿼리에 대한 정보를 검색합니다."""
    # 실제 구현에서는 실제 웹 검색 API를 사용할 수 있습니다
    # 여기서는 시뮬레이션 데이터를 반환합니다
    search_results = {
        "history of the web": {
            "timeline": "1989년 팀 버너스리가 WWW 제안, 1990년 첫 웹 브라우저 개발, 1993년 모자이크 브라우저 출시",
            "key_figures": "팀 버너스리, 마크 안드리슨",
            "technologies": "HTML, HTTP, URL"
        },
        "web development": {
            "evolution": "정적 웹 → 동적 웹 → 웹 2.0 → 모바일 웹 → 현대 웹",
            "frameworks": "React, Vue, Angular"
        }
    }
    
    # 검색 결과를 기반으로 답변 생성
    if "history" in query.lower() and "web" in query.lower():
        return f"검색 결과: {search_results['history of the web']}"
    elif "web" in query.lower():
        return f"검색 결과: {search_results['web development']}"
    else:
        return f"'{query}'에 대한 일반적인 검색 결과를 찾았습니다."

async def record_notes(ctx: Context, topic: str, content: str) -> str:
    """연구 노트를 기록하고 상태에 저장합니다."""
    state = await ctx.store.get("state")
    
    if "research_notes" not in state:
        state["research_notes"] = {}
    
    # 노트 추가
    state["research_notes"][topic] = content
    await ctx.store.set("state", state)
    
    return f"'{topic}' 주제에 대한 노트가 기록되었습니다: {content[:100]}..."

async def write_report(ctx: Context, topic: str) -> str:
    """기록된 노트를 바탕으로 마크다운 리포트를 작성합니다."""
    state = await ctx.store.get("state")
    
    research_notes = state.get("research_notes", {})
    
    if not research_notes:
        return "리포트를 작성하기 위한 연구 노트가 없습니다. 먼저 연구를 진행해주세요."
    
    # 마크다운 리포트 생성
    report = f"# {topic} 리포트\n\n"
    
    for note_topic, note_content in research_notes.items():
        report += f"## {note_topic}\n\n"
        report += f"{note_content}\n\n"
    
    report += "## 결론\n\n"
    report += "이 연구를 통해 웹의 역사와 발전 과정을 살펴보았습니다.\n"
    
    # 상태에 리포트 저장
    state["report_content"] = report
    await ctx.store.set("state", state)
    
    return f"'{topic}' 리포트가 작성되었습니다:\n\n{report}"

async def review_report(ctx: Context) -> str:
    """작성된 리포트를 검토하고 피드백을 제공합니다."""
    state = await ctx.store.get("state")
    
    report_content = state.get("report_content", "")
    
    if not report_content or report_content == "Not written yet.":
        return "검토할 리포트가 없습니다. 먼저 리포트를 작성해주세요."
    
    # 리포트 검토
    word_count = len(report_content.split())
    section_count = report_content.count("##")
    
    feedback = []
    
    if word_count < 100:
        feedback.append("리포트가 너무 짧습니다. 더 자세한 내용을 추가해주세요.")
    
    if section_count < 2:
        feedback.append("리포트에 더 많은 섹션을 추가하여 구조를 개선해주세요.")
    
    if not feedback:
        feedback.append("리포트가 잘 작성되었습니다. 좋은 구조와 내용을 가지고 있습니다.")
    
    review_text = "\n".join(feedback)
    
    # 상태에 검토 결과 저장
    state["review"] = review_text
    await ctx.store.set("state", state)
    
    return f"리포트 검토 완료:\n\n{review_text}"


In [53]:
from llama_index.core.agent.workflow import AgentWorkflow, FunctionAgent

# --- create our specialist agents ------------------------------------------------
research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Search the web and record notes.",
    system_prompt="You are a researcher… hand off to WriteAgent when ready.",
    llm=llm,
    tools=[search_web, record_notes],
    can_handoff_to=["WriteAgent"],
)

write_agent = FunctionAgent(
    name="WriteAgent",
    description="Writes a markdown report from the notes.",
    system_prompt="You are a writer… ask ReviewAgent for feedback when done.",
    llm=llm,
    tools=[write_report],
    can_handoff_to=["ReviewAgent", "ResearchAgent"],
)

review_agent = FunctionAgent(
    name="ReviewAgent",
    description="Reviews a report and gives feedback.",
    system_prompt="You are a reviewer…",  # etc.
    llm=llm,
    tools=[review_report],
    can_handoff_to=["WriteAgent"],
)

# --- wire them together ----------------------------------------------------------
agent_workflow = AgentWorkflow(
    agents=[research_agent, write_agent, review_agent],
    root_agent=research_agent.name,
    initial_state={
        "research_notes": {},
        "report_content": "Not written yet.",
        "review": "Review required.",
    },
)

resp = await agent_workflow.run(
    user_msg="Write me a report on the history of the web …"
)
print(resp)

The review of the report on the history of the web indicates that it is too short and requires more detailed content. Would you like me to assist in expanding the report with additional information?


In [54]:
# 워크플로우 상태 확인
from llama_index.core.workflow import Context

ctx = Context(agent_workflow)
await agent_workflow.run(user_msg="웹의 역사에 대한 리포트를 작성해주세요.", ctx=ctx)

# 최종 상태 확인
final_state = await ctx.store.get("state")
print("\\n=== 최종 상태 ===")
print(f"연구 노트 개수: {len(final_state.get('research_notes', {}))}")
print(f"리포트 길이: {len(final_state.get('report_content', ''))}")
print(f"검토 상태: {final_state.get('review', 'No review')}")

# 연구 노트 확인
print("\\n=== 연구 노트 ===")
for topic, content in final_state.get('research_notes', {}).items():
    print(f"주제: {topic}")
    print(f"내용: {content}")
    print("-" * 50)

# 최종 리포트 확인
print("\\n=== 최종 리포트 ===")
print(final_state.get('report_content', 'No report generated'))


\n=== 최종 상태 ===
연구 노트 개수: 3
리포트 길이: 546
검토 상태: 리포트가 너무 짧습니다. 더 자세한 내용을 추가해주세요.
\n=== 연구 노트 ===
주제: history of the web
내용: - 1989: Tim Berners-Lee proposed the World Wide Web.
- 1990: The first web browser was developed.
- 1993: The Mosaic browser was released.
--------------------------------------------------
주제: key figures in the history of the web
내용: - Tim Berners-Lee: Inventor of the World Wide Web.
- Marc Andreessen: Co-author of Mosaic, the first widely used web browser.
--------------------------------------------------
주제: technologies in the history of the web
내용: - HTML (HyperText Markup Language)
- HTTP (HyperText Transfer Protocol)
- URL (Uniform Resource Locator)
--------------------------------------------------
\n=== 최종 리포트 ===
# history of the web 리포트

## history of the web

- 1989: Tim Berners-Lee proposed the World Wide Web.
- 1990: The first web browser was developed.
- 1993: The Mosaic browser was released.

## key figures in the history of the web

- Tim Berners-

# Orchestration Pattern

In [ ]:
import re
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.workflow import Context

# assume research_agent / write_agent / review_agent defined as before
# except we really only need the `search_web` tool at a minimum


async def call_research_agent(ctx: Context, prompt: str) -> str:
    """Useful for recording research notes based on a specific prompt."""
    result = await research_agent.run(
        user_msg=f"Write some notes about the following: {prompt}"
    )

    state = await ctx.store.get("state")
    state["research_notes"].append(str(result))
    await ctx.store.set("state", state)

    return str(result)


async def call_write_agent(ctx: Context) -> str:
    """Useful for writing a report based on the research notes or revising the report based on feedback."""
    state = await ctx.store.get("state")
    notes = state.get("research_notes", None)
    if not notes:
        return "No research notes to write from."

    user_msg = f"Write a markdown report from the following notes. Be sure to output the report in the following format: <report>...</report>:\n\n"

    # Add the feedback to the user message if it exists
    feedback = state.get("review", None)
    if feedback:
        user_msg += f"<feedback>{feedback}</feedback>\n\n"

    # Add the research notes to the user message
    notes = "\n\n".join(notes)
    user_msg += f"<research_notes>{notes}</research_notes>\n\n"

    # Run the write agent
    result = await write_agent.run(user_msg=user_msg)
    report = re.search(r"<report>(.*)</report>", str(result), re.DOTALL).group(
        1
    )
    state["report_content"] = str(report)
    await ctx.store.set("state", state)

    return str(report)


async def call_review_agent(ctx: Context) -> str:
    """Useful for reviewing the report and providing feedback."""
    state = await ctx.store.get("state")
    report = state.get("report_content", None)
    if not report:
        return "No report content to review."

    result = await review_agent.run(
        user_msg=f"Review the following report: {report}"
    )
    state["review"] = result
    await ctx.store.set("state", state)

    return result


orchestrator = FunctionAgent(
    system_prompt=(
        "You are an expert in the field of report writing. "
        "You are given a user request and a list of tools that can help with the request. "
        "You are to orchestrate the tools to research, write, and review a report on the given topic. "
        "Once the review is positive, you should notify the user that the report is ready to be accessed."
        "should end before 30sec"
    ),
    llm=OpenAI(model="gpt-4o-mini"),
    tools=[
        call_research_agent,
        call_write_agent,
        call_review_agent,
    ],
    initial_state={
        "research_notes": [],
        "report_content": None,
        "review": None,
    },
    timeout=60,
)

response = await orchestrator.run(
    user_msg="Write me a report on the history of the web …"
)
print(response)